In [72]:
import pandas as pd
import numpy as np

import warnings
from tqdm import TqdmExperimentalWarning

warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt
from traffic.core import Traffic

warnings.filterwarnings('ignore')

from metpy.calc import wind_speed
from metpy.calc import wind_direction
from metpy.units import units

from multiprocessing import process

import multiprocessing as mp
from tqdm import tqdm

import os
import shutil

from pandarallel import pandarallel

from pathlib import Path

import glob


In [73]:
data = pd.read_csv("data/datasets_with_regression.csv")
len(data)

527162

In [74]:
data["takeoff_time"] = pd.to_datetime(data["actual_offblock_time"]) + pd.to_timedelta(data['taxiout_time'], unit='min')
data["arrival_time"] = pd.to_datetime(data["arrival_time"]) 

In [75]:
## function vs pvlib 

def alt2pres(altitude):
    '''
    Determine site pressure from altitude.

    Parameters
    ----------
    altitude : numeric
        Altitude above sea level. [m]

    Returns
    -------
    pressure : numeric
        Atmospheric pressure. [Pa]

    Notes
    ------
    The following assumptions are made

    ============================   ================
    Parameter                      Value
    ============================   ================
    Base pressure                  101325 Pa
    Temperature at zero altitude   288.15 K
    Gravitational acceleration     9.80665 m/s^2
    Lapse rate                     -6.5E-3 K/m
    Gas constant for air           287.053 J/(kg K)
    Relative Humidity              0%
    ============================   ================

    References
    -----------
    .. [1] "A Quick Derivation relating altitude to air pressure" from
       Portland State Aerospace Society, Version 1.03, 12/22/2004.
    '''

    press = 100 * ((44331.514 - altitude) / 11880.516) ** (1 / 0.1902632)

    return press


# from Yoshiki443/weather_parameters

# Calculate cross wind, tail wind, and head wind component of wind
# given wind velocity[m/s, kt, etc], wind direction[degree],
# and runway direction[degree] of a airpot.

def cross_wind(wspd, wdir, rwy):
    return np.abs( wspd * np.sin( np.deg2rad(wdir - rwy) ) )

def tail_wind(wspd, wdir, rwy):
    return - wspd * np.cos( np.deg2rad(wdir - rwy) )

def head_wind(wspd, wdir, rwy):
    return - Tail_Wind(wspd, wdir, rwy)

In [76]:

def process_row(row):
    flight_id = row["flight_id"]
    takeoff = row["takeoff_time"]
    land = row["arrival_time"]
    type_code = row["replacer"]
    best_fuel = row["total_fuel_fill"]
    path = "/mnt/SMB_share/mark/flight_competition/repartitioned/first_five="+str(flight_id)[:5]+"/flight_id="+str(flight_id)+"/"
    my_file = Path(path)
    if my_file.exists():
        data = pd.read_parquet(path)
        data["icao24"] = flight_id
        data = data.rename(columns={"v_component_of_wind": "wind_v", "u_component_of_wind": "wind_u"})
        start_and_end = pd.DataFrame([[flight_id, takeoff, 0], [flight_id, land, 0]], columns=["icao24", "timestamp", "altitude"])
        data = pd.concat([data, start_and_end])
        data["sec_since_takeoff"] = pd.to_timedelta(pd.to_datetime(data["timestamp"], utc=True, unit="s") - takeoff).astype('timedelta64[s]') / np.timedelta64(1, 's')
        
        t = (Traffic(data)
                .filter()
                .resample('30s', how="interpolate")
                .eval()
        )
        
        g = t[0]
        g = g.phases()
        
        g = g.fuelflow(typecode=type_code)
        g = g.compute_TAS()
        
        max_fuel = g.fuel_max
        data = g.data

        ## find first two minutes of cruise
        data["is_cruise"] = np.where(data["phase"]=="CRUISE", 1, 0)
        data["is_cruise_four"] = data["is_cruise"].rolling(4).sum()
        is_four = data[data["is_cruise_four"]==4.0]
        if len(is_four) > 0: 
            first_cruise = is_four.iloc[0]["timestamp"]
            first_cruise_alt = is_four.iloc[0]["altitude"]
        else: 
            first_cruise = np.nan
            first_cruise_alt = np.nan

        data_trim = data[data["phase"]=="CLIMB"]
        
        #print(max_fuel)
        data_trim["wind_u"] = data_trim["wind_u"].astype(float)
        data_trim["wind_v"] = data_trim["wind_v"].astype(float)

        data_trim["wind_speed"] = data_trim.apply(lambda x: wind_speed(x["wind_u"] * units('m/s'), x["wind_v"] * units('m/s')).magnitude, axis=1)
        data_trim["wind_direction"] = data_trim.apply(lambda x: wind_direction(x["wind_u"] * units('m/s'), x["wind_v"] * units('m/s')).magnitude, axis=1)
        data_trim["tail_wind"] = data_trim.apply(lambda x: tail_wind(x["wind_speed"], x["wind_direction"], x["heading"]), axis=1)
        data_trim["cross_wind"] = data_trim.apply(lambda x: cross_wind(x["wind_speed"], x["wind_direction"], x["heading"]), axis=1)
        data_trim["pressure"] = data_trim.apply(lambda x: alt2pres(x["altitude"]), axis=1)
        data_trim["flight_id"] = flight_id
        
        #data_trim["max_fuel"] = max_fuel
        characteristics = pd.DataFrame([[flight_id, max_fuel, first_cruise, first_cruise_alt]], columns=["flight_id", "max_fuel", "first_cruise", "first_cruise_alt"])
        
        return data_trim, characteristics

    else: 
        return pd.DataFrame([]), pd.DataFrame([])


def apply_multiprocessing(df, func, num_processes):
    # Convert DataFrame rows into a list of dictionaries for easy processing
    df_split = [row for _, row in df.iterrows()]
    
    # Create a pool of workers
    with mp.Pool(processes=num_processes) as pool:
        # Map the function to the DataFrame rows
        results = list(tqdm(pool.imap(func, df_split), total=len(df_split)))

        charact_array = [x[1] for x in results]
        traject_array = [x[0] for x in results]

        characteristics_all = pd.concat(charact_array)
        traject_all = pd.concat(traject_array)        

    return characteristics_all, traject_all


In [77]:
chunks = np.array_split(data, 300)
print(len(chunks))

300


In [78]:
for i, chunk in enumerate(chunks): 
    print(i)
    characteristics_all, traject_all = apply_multiprocessing(chunk, process_row, 11)
    characteristics_all.to_csv("data/chunks/characteristics_all_rerun_"+str(i)+".csv", index=False)
    traject_all.to_csv("data/chunks/traject_all_rerun_"+str(i)+".csv", index=False)


0


100%|███████████████████████████████████████████████████████████████████████████████████████| 1758/1758 [02:04<00:00, 14.09it/s]


1


100%|███████████████████████████████████████████████████████████████████████████████████████| 1758/1758 [02:03<00:00, 14.19it/s]


2


...

100%|███████████████████████████████████████████████████████████████████████████████████████| 1757/1757 [02:01<00:00, 14.44it/s]


298


100%|███████████████████████████████████████████████████████████████████████████████████████| 1757/1757 [02:04<00:00, 14.07it/s]


299


100%|███████████████████████████████████████████████████████████████████████████████████████| 1757/1757 [02:23<00:00, 12.28it/s]


<!-- False     14681
True     460291 -->